In [1]:
#import basic modules
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
#import peak_utils

We will now load in all models so we can check which parts are significant. Only an example subset is stored on the github repo. You can download this data from the github repo or as a .tar.gz file from the website under the datasets section.


In [2]:
#this command will load in all datasets, each is a numpy pickle file.
allnames = glob.glob('../MCMC/*.npy')

In [3]:
allnames

[]

In [4]:
#we set an averaging size of 15, as this is a typical size for a binding site.
windowsize = 15
def do_sum2(s):
    '''this function does a summation 15 base pairs from the experession shifts models.
    We will be seeing if the summation of 15 consecutive base pairs are significant for gene
    expression (99% confidence interval).'''
    outarr = np.zeros((9000,160 - windowsize))
    for i in range(160-windowsize):
        outarr[:,i] = s[:,i:(i+windowsize)].sum(axis=1)
    return outarr

#we define some column names. The only purpose of that these are the columns that are used in 
#the any energy matrices.
val_cols = ['val_A','val_C','val_G','val_T']


    
for name in allnames:
    try:
        #we will use pymc to load in the database of all MCMC steps.
        em = np.load(name)
        
        
        #we have removed 1000 burnin steps. Due to the thinning of 60, this is
        #actually 60000 steps.
        emat = em[:,:]

        #sum into groupings of 15 base pairs so that we can see if large regions are statistically
        #signficant for expression.
        summedarr2 = do_sum2(emat)

        #initialize array where we will store whether or not the outcome is signficant.
        is_significant = np.zeros(160-windowsize)
        for i in range(160-windowsize):
            #make a 99.5 percent confidence interval
            #to do this we will check from .5 to .95 because we want to know if 0 
            #is in the range (.5% to 100%) or (0 to 99.5%) range, depending on
            #whether the shift is positive or negative.
            CI = np.percentile(summedarr2[:,i],[.5,99.5])
            #if zero is in interval, the base is not signficant, otherwise it is.
            if CI[0] < 0 and CI[1] > 0:
                pass
            elif CI[0] > 0:
                is_significant[i] = 1
            else:
                is_significant[i] = -1
        #both plot and save signficance results. signficant sites will end 15 base pairs after the last
        #shown signficant base because we average over 15 bases.
        fig,ax = plt.subplots()
        plt.bar(range(7,167-windowsize),is_significant)
        ax.set_xlabel('position')
        ax.set_ylabel('significant shift')
        plt.savefig(name.split('.npy')[0] + '_window_autocall_p1percentv2_' + str(windowsize),format='eps')
        np.savetxt(name.split('.npy')[0] + '_window_auto_p1percentv2_' + str(windowsize) + '.txt',is_significant)
        plt.clf()
    except Exception as e:
        print(e)

The resulting figure shows a 0 for bases that are not significant, 1 for bases that have a significant positive impact, and a -1 for bases that have a signficant negative impact. The results are also saved as a text file.